In [ ]:
import time
import os
from binance.client import Client
from dotenv import load_dotenv

# Get the path to the current directory
current_directory = os.getcwd()

# Specify the path to the .env file relative to the current directory
dotenv_path = os.path.join(current_directory, '.env')

# Load the environment variables from the .env file
load_dotenv(dotenv_path)

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret)

def has_open_positions():
    positions = client.futures_account()
    for position in positions['positions']:
        quantity = float(position['positionAmt'])
        if quantity != 0:
            return True
    print('No positions')
    return False
    
def close_all():
    positions = client.futures_account()

    for position in positions['positions']:
        symbol = position['symbol']
        quantity = float(position['positionAmt'])
        if float(quantity) != 0:
            print(symbol)
            
            ticker = client.futures_symbol_ticker(symbol=symbol)
            current_price = float(ticker['price'])
    
            if quantity > 0:
                side = 'SELL'
                positionSide = 'LONG'
            else:
                side = 'BUY'
                positionSide = 'SHORT'
    
            # Place limit order for exit at current market price
            order = client.futures_create_order(
                symbol=symbol,
                side=side,
                type='LIMIT',
                timeInForce='GTC',
                quantity=abs(quantity),
                price=current_price,
                positionSide=positionSide
            )
            print(order)

def close_all_loop():
    while has_open_positions():
        close_all()
        time.sleep(7)  # Adjust the interval as needed

In [ ]:
if __name__ == '__main__':
    close_all_loop()